## ตัวอย่างการทำ Text classification
เพื่อจำแนกเอกสารประเภท Spam Mail




ติดตั้ง Libraries สำหรับจัดการข้อมูลประเภทข้อความภาษาอังกฤษ

In [6]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk import  word_tokenize
from nltk.corpus import stopwords
import spacy
import string
import re


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/phutharesuanwachirapisut/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/phutharesuanwachirapisut/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


ทำการเชื่อม Google Drive

In [7]:
# from google.colab import drive 

# drive.mount('/content/gdrive')

ระบุ Folder ใน Drive ที่ต้องการใช้

In [8]:
# import os
# os.chdir("/content/gdrive/My Drive/Colab Notebooks")

อ่านข้อมูลจากไฟล์

In [9]:
import pandas as pd

data = pd.read_csv("spam.csv",encoding='latin-1')
print(data)

    Class                                            Content
0     ham     Not really dude, have no friends i'm afraid :(
1     ham                            Coffee cake, i guess...
2     ham  Merry Christmas to you too babe, i love ya *ki...
3     ham  Hey... Why dont we just go watch x men and hav...
4     ham  cud u tell ppl im gona b a bit l8 cos 2 buses ...
..    ...                                                ...
456  spam  This msg is for your mobile content order It h...
457  spam    You have 1 new message. Please call 08715205273
458  spam  December only! Had your mobile 11mths+? You ar...
459  spam  Get 3 Lions England tone, reply lionm 4 mono o...
460  spam       PRIVATE! Your 2003 Account Statement for 078

[461 rows x 2 columns]


# Data Preprocessing  สร้างเป็นฟังก์ชันเพื่อเรียกใช้ซ้ำ

In [10]:
def data_preprocessing(cleaned_data):
    # กำจัด URL
    cleaned_data = " ".join(re.sub(r"https?://[A-Za-z0-9./]+", '', w ) for w in cleaned_data.split()) 

    # แปลงข้อมูลเป็นตัวพิมพ์เล็ก ... lower case
    cleaned_data = " ".join(w.lower() for w in cleaned_data.split())

    # Lemmatization ด้วย Spacy
    nlp = spacy.load("en_core_web_sm")
    cleaned_data =  " ".join([w.lemma_ for w in nlp(cleaned_data) if w.lemma_ != "-PRON-"])

    # กำจัด stop words
    stop = stopwords.words('english')
    cleaned_data = " ".join(w for w in word_tokenize(cleaned_data) if w not in stop)

    # ตัดเครื่องหมายวรรคตอน ... remove punctuation
    cleaned_data =  " ".join([w for w in cleaned_data.split() if w not in list(string.punctuation)])

    # กำจัดตัวเลข
    cleaned_data = " ".join(re.sub(r"[0-9.]+%", '', w ) for w in cleaned_data.split()) 

    # กำจัดข้อความหลัง ' เครื่องหมาย aposophi
    cleaned_data = " ".join(re.sub(r"(')[A-Za-z]+", '', w ) for w in cleaned_data.split())

    return cleaned_data

เรียกใช้งานฟังก์ชัน data_preprocessing

In [ ]:
data['Content'] = data['Content'].apply(data_preprocessing)
# แสดงผลข้อมูลหลังจาก Data Preprocessing
print(data)

# สร้าง TF-IDF vectors

In [ ]:
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
encoder = preprocessing.LabelEncoder()
y = encoder.fit_transform(data['Class'])

tfidf_vect = TfidfVectorizer(max_features=5000)
tfidf_vect.fit(data['Content'])
x_tfidf = tfidf_vect.transform(data['Content'])
print(x_tfidf[0:5] )


สร้าง Training และ Testing set

In [ ]:
# Split the data into train and validation
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(x_tfidf, y)


In [ ]:
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
# Model building and evaluation
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
models = []
models.append(('NB', MultinomialNB()))
models.append(('Log-reg', LogisticRegression()))
models.append(('SVC', SVC()))
models.append(('RandomForest', RandomForestClassifier()))


# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name, clf in models:
  model = clf
  model.fit(X_train,y_train)
  Predited_class = model.predict(X_test)
  accuracy=accuracy_score(y_test, Predited_class)
  results.append(accuracy)
  names.append(name)
  msg = "%s: %f " % (name, accuracy)
  print(msg)
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.bar(names,results)

plt.show()

# Apply the best model

In [ ]:
model = MultinomialNB()
model.fit(X_train,y_train)

Appication form for data classifcaion 

In [ ]:
#@title Enter Content
Content = "I  do not like it" #@param {type:"string"}
# use the classifier to predict the sentiment of given text
Content = data_preprocessing(Content)
xtest_tfidf = tfidf_vect.transform([Content])
result = model.predict(xtest_tfidf)

result = encoder.inverse_transform(result)
if result[0] == 'spam':
      print("This is a spam message.")
elif result[0] == 'ham':
      print("This is not a spam message.")
